In [19]:
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.preprocessing import StandardScaler

from util.create_architecture import make_dense_model

In [48]:
# Define a function to load the data in
data_path = "data/"

def load_single_mean(lookback_game):
    X = np.load(data_path + f"single_mean_lg{lookback_game}_X.npy")
    Y = np.load(data_path + f"single_mean_lg{lookback_game}_Y.npy")
    ids = np.load(data_path + f"single_mean_lg{lookback_game}_ids.npy")
    return X, Y, ids

def load_single_diff(lookback_game):
    X = np.load(data_path + f"single_diff_lg{lookback_game}_X.npy")
    Y = np.load(data_path + f"single_diff_lg{lookback_game}_Y.npy")
    ids = np.load(data_path + f"single_diff_lg{lookback_game}_ids.npy")
    return X, Y, ids

def load_double_mean(lookback_game):
    X = np.load(data_path + f"double_mean_lg{lookback_game}_X.npy")
    Y = np.load(data_path + f"double_mean_lg{lookback_game}_Y.npy")
    ids = np.load(data_path + f"double_mean_lg{lookback_game}_ids.npy")
    return X, Y, ids

def load_double_diff(lookback_game):
    X = np.load(data_path + f"single_diff_lg{lookback_game}_X.npy")
    Y = np.load(data_path + f"single_diff_lg{lookback_game}_Y.npy")
    ids = np.load(data_path + f"single_diff_lg{lookback_game}_ids.npy")
    return X, Y, ids

In [52]:
# Define a function for creating and training the model
def eval_models(X, Y, ids, n_models, is_single, model_name):
    
#     learning_rate = 2e-4
#     epochs = 200 # Should always stop early.
    
    train_pct, test_pct, val_pct = 0.6, 0.2, 0.2
    
    # Split the data
    X_train = X[:int(train_pct*X.shape[0])]
    X_test = X[int(train_pct*X.shape[0]):int((train_pct+test_pct)*X.shape[0])]
    X_val = X[-int(val_pct*X.shape[0]):]
    
    Y_train = Y[:int(train_pct*X.shape[0])]
    Y_test = Y[int(train_pct*X.shape[0]):int((train_pct+test_pct)*X.shape[0])]
    Y_val = Y[-int(val_pct*X.shape[0]):]
    
    ids_train = ids[:int(train_pct*X.shape[0])]
    ids_test = ids[int(train_pct*X.shape[0]):int((train_pct+test_pct)*X.shape[0])]
    ids_val = ids[-int(val_pct*X.shape[0]):]
    
    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_val = scaler.transform(X_val)
    
    # Loop through and train models
    trained_models = []
    for _ in range(n_models):
        
        # Define the model structure
        model = make_dense_model(input_size=X.shape[1], hidden_layer_sizes=[10], 
                                 regularization=None, activation="relu")
        
        # Compile the model
        loss = tf.keras.losses.BinaryCrossentropy()
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        metrics = ['accuracy']
        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        # Fit the model
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)]
        model.fit(x=X_train, y=Y_train, validation_data=(X_val, Y_val), 
                  batch_size=64, epochs=epochs, callbacks=callbacks, verbose=0)
        
        trained_models.append(model)
        
        # Print out a quick evaluation
        val_acc = model.evaluate(X_val, Y_val, return_dict=True, verbose=0)['accuracy']
        print(f"Validation Accuracy: {val_acc*100:.2f}%")
        
    # Evaluate all the models together
    if is_single:
        final_accuracy = evaluate_single_models(trained_models, X_val, Y_val, ids_val)
    else:
        final_accuracy = evaluate_double_models(trained_models, X_val, Y_val)
        
    print(f"{model_name} Final Accuracy: {final_accuracy*100:.2f}%.")
    return final_accuracy
        

    
def evaluate_single_models(models, X, Y, ids):
    
    # Generate predictions
    predictions = np.array([m.predict(X, verbose=0).reshape(-1) for m in models])
    predictions = predictions.mean(axis=0) > 0.500
    
    # Use ids to find the same games
    right, wrong = 0, 0
    for game_id in np.unique(ids):
        game_mask = ids == game_id
        
        if np.sum(game_mask) != 2:
            continue
        
        game_preds = predictions[game_mask]
        game_Y = Y[game_mask]
        
        if game_Y[np.argmax(game_preds)]:
            right += 1
        else:
            wrong += 1
            
    accuracy = right / (right + wrong)
    return accuracy
    
def evaluate_double_models(models, X, Y):
    
    # Generate predictions
    predictions = np.array([m.predict(X, verbose=0).reshape(-1) for m in models])
    predictions = predictions.mean(axis=0) > 0.500
    
    accuracy = np.mean(predictions == Y)
    return accuracy
        

In [53]:
# Define the testing parameters
lookback_games = [1]
n_models = 1

learning_rate = 2e-4
epochs = 200 # Should always stop early.

# Loop through the data and store the accuracies
all_accuracies = np.zeros((len(lookback_games), 4+1))
for i, lookback_game in enumerate(lookback_games):
    
    # Single Mean Data
    X, Y, ids = load_single_mean(lookback_game)
    single_mean_accuracy = eval_models(X, Y, ids, n_models, is_single=True, model_name="Single Mean")
    
    # Single Diff Data
    X, Y, ids = load_single_diff(lookback_game)
    single_diff_accuracy = eval_models(X, Y, ids, n_models, is_single=True, model_name="Single Diff")
    
    # Double Mean Data
    X, Y, ids = load_double_mean(lookback_game)
    double_mean_accuracy = eval_models(X, Y, ids, n_models, is_single=False, model_name="Double Mean")
    
    # Double Diff Data
    X, Y, ids = load_double_diff(lookback_game)
    double_diff_accuracy = eval_models(X, Y, ids, n_models, is_single=False, model_name="Double Diff")
    
    # Track them (first column stores the lookback game value)
    all_accuracies[i] = [lookback_game, single_mean_accuracy, single_diff_accuracy, double_mean_accuracy, 
                         double_diff_accuracy]
    
# Save the tracking array
np.save("results/data_type_results.npy", all_accuracies)

Epoch 40: early stopping
Validation Accuracy: 52.99%
Single Mean Final Accuracy: 50.97%.
Epoch 76: early stopping
Validation Accuracy: 52.79%
Single Diff Final Accuracy: 51.89%.
Epoch 63: early stopping
Validation Accuracy: 53.39%
Double Mean Final Accuracy: 53.39%.
Epoch 45: early stopping
Validation Accuracy: 52.96%
Double Diff Final Accuracy: 52.96%.
